# Memory

>

In [ ]:
#| default_exp memory

In [ ]:
#| export
import datetime, uuid, dspy
from typing import List, Optional
from cogitarelink.core.graph import GraphManager
from cogitarelink.core.entity import Entity
from cogitarelink.reason.prov import wrap_patch_with_prov

REFLECTION_GRAPH = "urn:agent:reflections"
REFLECTION_TYPE  = "https://w3id.org/cogitarelink#ReflectionNote"

class ReflectionStore:
    """Persist 'lesson learned' notes as JSON-LD entities in the Cogitarelink graph."""
    def __init__(self, graph: GraphManager):
        self.graph = graph

    def add(self, text: str, tags: Optional[List[str]] = None) -> str:
        note_id = f"urn:uuid:{uuid.uuid4()}"
        now     = datetime.datetime.utcnow().isoformat()
        content = {
            "@id": note_id,
            "@type": REFLECTION_TYPE,
            "text": text,
            "tags": tags or [],
            "dateCreated": now
        }
        ent = Entity(vocab=["clref","schema"], content=content)
        with wrap_patch_with_prov(
            self.graph, source="urn:agent:self",
            agent="urn:agent:self", activity="urn:agent:addReflection"
        ):
            self.graph.ingest_entity(ent)
        return note_id

    def retrieve(self, limit: int = 5, tag_filter: Optional[str] = None) -> List[Entity]:
        """Fetch up to `limit` most recent notes, optionally filtering by tag."""
        ids = []
        triples = self.graph.query(
            pred="http://schema.org/dateCreated",
            graph_id=REFLECTION_GRAPH
        )
        triples.sort(key=lambda t: t[2], reverse=True)
        for s,_,_ in triples[:limit]:
            if tag_filter:
                tag_triples = self.graph.query(
                    subj=s, pred="http://schema.org/tags", graph_id=REFLECTION_GRAPH
                )
                tags = [o for (_,_,o) in tag_triples]
                if tag_filter not in tags:
                    continue
            ids.append(s)
        ents = []
        for nid in ids:
            t = self.graph.query(
                subj=nid, pred="http://schema.org/text", graph_id=REFLECTION_GRAPH
            )
            text = t[0][2] if t else ""
            ents.append(Entity(vocab=["clref","schema"], content={
                "@id": nid,
                "@type": REFLECTION_TYPE,
                "text": text
            }))
        return ents

    def as_prompt(self, limit: int = 5) -> str:
        notes = self.retrieve(limit)
        return "\n".join(f"• {e.content['text']}" for e in notes)

/Users/cvardema/dev/git/LA3D/cogitarelink/cogitarelink-dspy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()